### DataShaping
研究に使用するデータの成型を行う

In [1]:
import pandas as pd
import numpy as np
import re
import os

import MDUS

In [4]:
#　rotB電流のデータの成型
rotB_original = pd.read_csv('./data/deltaBcurrent.csv',index_col=0)
rotB_original.to_pickle('./data/other/rotBcurrent.pkl')

In [13]:
# Mercury まわりの成型
rss = pd.read_csv('./data/Mercury/Rss.csv',index_col=0)
TA = pd.read_csv('./data/Mercury/TrueAnomaly.csv',index_col=0)

result = pd.merge(TA,rss,left_index=True,right_index=True)
result.to_pickle('./data/Mercury/MercuryData.pkl')

In [6]:
# dip classificationのデータ成型
data = pd.read_csv('./data/Dip/DipClassification.csv',index_col=0)
dip_type = []
for i in data.itertuples():
    left = i[1]
    right = i[2]
    if left/right > 1:
        dip_type.append('dip')
    else:
        dip_type.append('ns')
data['type'] = dip_type
data.to_pickle('./data/Dip/DipClassification.pkl')

In [58]:
data = MDUS.MagData()
data.Input(orbit=52)
data.value

,X_MSO,Y_MSO,Z_MSO,Bx,By,Bz,|B|
date,,,,,,,
2011-04-13 04:19:21,0.769554,-0.291321,1.208715,-64.191,-2.953,42.774,77.193392
2011-04-13 04:19:22,0.768457,-0.290633,1.208897,-50.785,-2.117,35.357,61.917004
2011-04-13 04:19:23,0.767358,-0.289945,1.209077,-32.251,-5.801,24.785,41.086188
2011-04-13 04:19:24,0.766260,-0.289257,1.209257,-41.991,-12.007,34.140,55.434211
2011-04-13 04:19:25,0.765160,-0.288569,1.209437,-42.896,-9.148,33.695,55.309183
...,...,...,...,...,...,...,...
2011-04-13 05:31:09,-1.578468,0.617373,-2.378999,-14.508,-26.114,-5.312,30.342057
2011-04-13 05:31:10,-1.578383,0.617208,-2.379799,-10.763,-25.421,-2.712,27.738499
2011-04-13 05:31:11,-1.578299,0.617043,-2.380599,-18.329,-10.427,2.325,21.215094


In [12]:
# scanデータの成型
input_path = '/home/togawa/data/messenger/fips/cdr/scan'
output_path = '/data/togawa/messenger/fips/cdr/scan'
EQTAB = [ 0.0464,  0.0511,  0.0564,  0.062 ,  0.0683,  0.0754, 0.0829,
         0.0913,  0.1004,  0.1107,  0.1219,  0.134 ,  0.1478, 0.1627,
         0.1789,  0.197 ,  0.217 ,  0.2388,  0.2631,  0.2896, 0.3189,
         0.351 ,  0.3863,  0.4255,  0.4682,  0.5156,  0.5677, 0.6251,
         0.688 ,  0.7576,  0.8343,  0.9184,  1.011 ,  1.1133, 1.2255,
         1.3493,  1.4855,  1.6358,  1.8007,  1.9828,  2.183 , 2.4034,
         2.6459,  2.9131,  3.2074,  3.531 ,  3.8877,  4.2802, 4.7126,
         5.1884,  5.7121,  6.2892,  6.9243,  7.6233,  8.393 , 9.2407,
        10.1738, 11.2011, 12.3322, 13.5774, 14.9482, 16.4578, 18.1198,
        19.9492]  
EQTAB.reverse()
data_types = ['start','stop','VE','proton','EP']  
files_file = [
    f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))
]

count = 0
for file in files_file:
    count += 1
    print(str(count)+'/'+str(len(files_file)))
    print(os.path.isfile(output_path + '/' + file))
    
    tmp = pd.read_pickle(input_path + '/' + file)
    result = pd.DataFrame()
    date = []
    for i in tmp[1].values:
        m = re.match(r'(\d{4})-(\d{3})T(\d{2}):(\d{2}):(\d{2}).(.*)',i)
        year = int(m.group(1))
        day = int(m.group(2))
        hour = int(m.group(3))
        minute = int(m.group(4))
        second = int(m.group(5))
        msecond = int(m.group(6))
        a = pd.Timedelta(days=day-1, hours=hour, minutes=minute, seconds=second, milliseconds=msecond)
        date.append(pd.to_datetime(year*1000 + 1, format='%Y%j') + a)
    for i in range(len(data_types)):
        result = pd.DataFrame()
        for j in range(len(EQTAB)):
            result[EQTAB[j]] = tmp.values[:,4+j+i*len(EQTAB)]
            m = re.match(r'(\d{4})_(\d{3}).pkl',file)
            outfile = m.group(1) + '_' + m.group(2) + '_' + data_types[i] + '.pkl'
        print(outfile)
        result['date'] = date
        result = result.set_index('date')
        result.to_pickle(output_path + '/' + outfile)

1/1566
False
2013_121_start.pkl
2013_121_stop.pkl
2013_121_VE.pkl
2013_121_proton.pkl
2013_121_EP.pkl
2/1566
False
2011_001_start.pkl
2011_001_stop.pkl
2011_001_VE.pkl
2011_001_proton.pkl
2011_001_EP.pkl
3/1566
False
2012_001_start.pkl
2012_001_stop.pkl
2012_001_VE.pkl
2012_001_proton.pkl
2012_001_EP.pkl
4/1566
False
2011_002_start.pkl
2011_002_stop.pkl
2011_002_VE.pkl
2011_002_proton.pkl
2011_002_EP.pkl
5/1566
False
2011_003_start.pkl
2011_003_stop.pkl
2011_003_VE.pkl
2011_003_proton.pkl
2011_003_EP.pkl
6/1566
False
2011_004_start.pkl
2011_004_stop.pkl
2011_004_VE.pkl
2011_004_proton.pkl
2011_004_EP.pkl
7/1566
False
2011_005_start.pkl
2011_005_stop.pkl
2011_005_VE.pkl
2011_005_proton.pkl
2011_005_EP.pkl
8/1566
False
2011_006_start.pkl
2011_006_stop.pkl
2011_006_VE.pkl
2011_006_proton.pkl
2011_006_EP.pkl
9/1566
False
2011_007_start.pkl
2011_007_stop.pkl
2011_007_VE.pkl
2011_007_proton.pkl
2011_007_EP.pkl
10/1566
False
2011_008_start.pkl
2011_008_stop.pkl
2011_008_VE.pkl
2011_008_proton

In [7]:
tmp = "2013_121.pkl"
print(tmp)
m = re.match(r'(\d{4})_(\d{3}).pkl',tmp)
print(m.group(1))
print(m.group(2))

2013_121.pkl
2013
121


In [5]:
result.columns

Index([19.9492, 18.1198, 16.4578, 14.9482, 13.5774, 12.3322, 11.2011, 10.1738,
        9.2407,   8.393,  7.6233,  6.9243,  6.2892,  5.7121,  5.1884,  4.7126,
        4.2802,  3.8877,   3.531,  3.2074,  2.9131,  2.6459,  2.4034,   2.183,
        1.9828,  1.8007,  1.6358,  1.4855,  1.3493,  1.2255,  1.1133,   1.011,
        0.9184,  0.8343,  0.7576,   0.688,  0.6251,  0.5677,  0.5156,  0.4682,
        0.4255,  0.3863,   0.351,  0.3189,  0.2896,  0.2631,  0.2388,   0.217,
         0.197,  0.1789,  0.1627,  0.1478,   0.134,  0.1219,  0.1107,  0.1004,
        0.0913,  0.0829,  0.0754,  0.0683,   0.062,  0.0564,  0.0511,  0.0464],
      dtype='float64')

In [48]:
data_types = np.array(['start','stop','VE','proton','EP'])
fuga = 'pyto'
if len(np.where(data_types == fuga)[0]) == 0:
    print('There is no data')
else:
    print(np.where(data_types == fuga)[0][0])

# print(np.where(data_types == 'start')[0][0])
# print(np.where(data_types == 'proton')[0][0])
# print(np.where(data_types == 'hoge'))

There is no data


In [ ]:
# orbitデータの成型
orbit_original = pd.read_csv('./data/Orbit/orbits.csv',index_col=0)
orbit_original = orbit_original.drop('year',axis=1)
orbit_columns = orbit_original.columns
for i in orbit_columns:
    orbit_original[i] = pd.to_datetime(orbit_original[i])
orbit_original.to_pickle('./data/Orbit/orbits.pkl')

In [ ]:
# 正しく成形できているかの確認 -> OK
data = MDUS.MagData()
data.Input(orbit=199)
data.Plot()

In [37]:
# Dipデータの成型
dip_original = pd.read_csv('./data/Dip/DipData.csv',index_col=0)
dip_original = dip_original.drop('year',axis=1)
dip_original = dip_original.drop('outbound',axis=1)
dip_columns = dip_original.columns
for i in dip_columns:
    dip_original[i] = pd.to_datetime(dip_original[i])
dip_original.to_pickle('./data/Dip/DipData.pkl')